In [1]:
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:imageomics/bioclip')
tokenizer = open_clip.get_tokenizer('hf-hub:imageomics/bioclip')

In [2]:
image_path="./plant-image.jpg"

In [ ]:
from PIL import Image
import torch

image = preprocess_val(Image.open(image_path)).unsqueeze(0)